In [1]:
%%capture
!pip install numba

In [2]:
from numba import jit
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
data = pd.read_csv('/content/drive/MyDrive/WQU-Capstone-HFT/Dataset/Kaiko/L3_OrderBooks_Extract/L5Book.csv')
data = data[(data.bid_1_price != 0) & (data.ask_1_price != 0)].reset_index(drop=True)
data = data.replace({0:np.nan})

In [4]:
timestamp = pd.DatetimeIndex(data.timestamp).astype(np.int64)/1000000
timestamp = np.array(timestamp)

In [5]:
feat_data = pd.DataFrame(data.timestamp)
feat_data['mid_price'] = (data.bid_1_price + data.ask_1_price)/2
feat_data['spread_b1'] = feat_data.mid_price - data.bid_1_price
feat_data['spread_b2'] = feat_data.mid_price - data.bid_2_price
feat_data['spread_b3'] = feat_data.mid_price - data.bid_3_price
feat_data['spread_b4'] = feat_data.mid_price - data.bid_4_price
feat_data['spread_b5'] = feat_data.mid_price - data.bid_5_price
feat_data['spread_a1'] = feat_data.mid_price - data.ask_1_price
feat_data['spread_a2'] = feat_data.mid_price - data.ask_2_price
feat_data['spread_a3'] = feat_data.mid_price - data.ask_3_price
feat_data['spread_a4'] = feat_data.mid_price - data.ask_4_price
feat_data['spread_a5'] = feat_data.mid_price - data.ask_5_price
feat_data['numerator']    = 0
feat_data['denominator']  = 0
feat_data = feat_data.replace({np.nan:0})
data = data.replace({np.nan:0})

In [6]:
b = 50
for i in range(1,6):
  feat_data.numerator   += (b - feat_data[f'spread_b{i}']) * data[f'bid_{i}_vol']
  feat_data.numerator   -= (-b - feat_data[f'spread_a{i}']) * data[f'ask_{i}_vol']
  feat_data.denominator += (b - feat_data[f'spread_b{i}']) * data[f'bid_{i}_vol']
  feat_data.denominator += (-b - feat_data[f'spread_a{i}']) * data[f'ask_{i}_vol']
feat_data['obi'] = feat_data.numerator/feat_data.denominator

In [7]:
triple_label = pd.read_csv('/content/drive/MyDrive/WQU-Capstone-HFT/Dataset/Kaiko/L3_OrderBooks_Extract/TradeLabel_2s.csv')

In [8]:
dataset = pd.DataFrame(data.timestamp)
dataset['spread'] = data.ask_1_price - data.bid_1_price
dataset['weight'] = feat_data.obi
dataset['triple_label'] = triple_label['triple_label']
dataset.triple_label[dataset.triple_label==-1] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
split = round(len(dataset.index) * 0.75)
X = dataset.iloc[:, 1:3]
X_train = X.iloc[:split, :].values
y_train = dataset.iloc[:split, -1].values
X_test =  X.iloc[split:, :].values
y_test =  dataset.iloc[split:, -1].values

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
import tensorflow as tf

In [12]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))
ann.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [14]:
ann.fit(X_train, y_train, batch_size = 1024, epochs = 100)

Epoch 1/100
715/715 [==============================] - 2s 2ms/step - loss: 0.5015 - accuracy: 0.7871
Epoch 2/100
715/715 [==============================] - 1s 2ms/step - loss: 0.4369 - accuracy: 0.8309
Epoch 3/100
715/715 [==============================] - 1s 2ms/step - loss: 0.4355 - accuracy: 0.8310
Epoch 4/100
715/715 [==============================] - 1s 2ms/step - loss: 0.4341 - accuracy: 0.8312
Epoch 5/100
715/715 [==============================] - 1s 2ms/step - loss: 0.4339 - accuracy: 0.8309
Epoch 6/100
715/715 [==============================] - 1s 2ms/step - loss: 0.4335 - accuracy: 0.8307
Epoch 7/100
715/715 [==============================] - 1s 2ms/step - loss: 0.4329 - accuracy: 0.8305
Epoch 8/100
715/715 [==============================] - 1s 2ms/step - loss: 0.4318 - accuracy: 0.8312
Epoch 9/100
715/715 [==============================] - 1s 2ms/step - loss: 0.4318 - accuracy: 0.8309
Epoch 10/100
715/715 [==============================] - 1s 2ms/step - loss: 0.4311 - accura

In [15]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[  2027 241895]
 [     0      0]]


0.008310033535310467

In [17]:
y_pred2 = ann.predict(X_train)
y_pred2 = (y_pred2 > 0.5)

In [18]:
cm = confusion_matrix(y_train, y_pred2)
print(cm)
accuracy_score(y_train, y_pred2)

[[  1925 121996]
 [  1874 605973]]


0.830725038536804

In [19]:
len(y_train)-y_train.sum()

123921